In [58]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [59]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [60]:
with open("vaccinated.json", "rt") as INFILE:
    data=json.load(INFILE)

In [61]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [62]:
def wrangle_data(rawdata):
    
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    vaccinateddf=pd.DataFrame(index=index, columns=['cases', 'vaccine'])
    
    for entry in datalist: 
        date=parse_date(entry['date'])
    for column in ['cases', 'vaccine']:
        
        if pd.isna(vaccinateddf.loc[date, column]): 
            
            value= float(entry[column]) if entry[column]!=None else 0.0
            
            vaccinateddf.loc[date, column]=value
            

    vaccinateddf.fillna(0.0, inplace=True)
            
    return vaccinateddf

vaccinateddf=wrangle_data(data) 



In [63]:

def access_api(button):
    
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)


apibutton.on_click(access_api)


display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [68]:
# in the dashboard, this should be computed from the json files
vaccinateddf=pd.read_pickle("vaccinateddf.pkl")

# Number of Cases compared to number of people vaccinated

In [65]:
series=wdg.SelectMultiple(
    options=['cases', 'vaccine'],
    value=['cases', 'vaccine'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],

    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def vaccinated_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vaccinateddf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(vaccinated_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

In [57]:

ctrls=wdg.VBox([series, scale])

form=wdg.HBox([graph, ctrls])

display(form)

# Created by Nur Baba-Ahmed with credit to Fabrizio Smeraldi

# Source: Based on UK Government data published by Public Health England.